# In questo documento andiamo a selezionare le feature singolarmente da ogni documento

In [1]:
# Importiamo le librerie

from os import listdir
from os.path import isfile, join
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
import pickle

In [2]:
# Variabili globali

mypath = 'dataset'

# Indice per il file corrente da analizzare 

current = 9

# Questi campi non sono di interesse, quindi li escludiamo dalle analisi

first = []

if current == 3:
    first = ['Dst Port', 'Protocol', 'Timestamp','Flow ID','Src IP','Dst IP']
else:
    first = ['Dst Port', 'Protocol', 'Timestamp']


In [3]:
# Mi vado a prendere i path di tutti i file nella cartella dataset

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [4]:
# Mi creo una cartella image all'interno del quale per ogni file vado a inserire le immagini d'interesse

if not os.path.exists("image"):
    os.makedirs("image")
    for a in onlyfiles:
        b = a.replace('.csv','')
        os.makedirs("image/"+b)
        os.makedirs("image/"+b+"/scatter")
        os.makedirs("image/"+b+"/istogrammi")

# Se non esiste, creo una cartella dove salvare le migliori feature

if not os.path.exists("top_feature"):
    os.makedirs("top_feature")
    for a in onlyfiles:
        b = a.replace('.csv','')
        os.makedirs("top_feature/"+b)

if not os.path.exists("pickle"):
    os.makedirs("pickle")
    for a in onlyfiles:
        b = a.replace('.csv','')
        os.makedirs("pickle/"+b)

# Per ogni documento andiamo a eseguire le operazioni di pre-processing, e a selezionare le feature d'interesse

In [5]:
document= pd.read_csv(mypath+'/'+onlyfiles[current])
scarto = []
scarto.append([])
folder = onlyfiles[current].replace('.csv','')
attaccanti_label = document['Label'].unique()[document['Label'].unique() != 'Label']

C:\Users\pc\AppData\Local\Temp\ipykernel_15328\3663514834.py:1: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  document= pd.read_csv(mypath+'/'+onlyfiles[current])


In [6]:
# Andiamo a eliminare i primi 3 campi inutili

for a in first:
    document.drop(columns=a,axis=1,inplace=True)

In [7]:
# Analisi della varianza, eliminiamo tutte le feature che hanno un solo valore

for a in document.head(0):
    
    if(document[a].unique().shape[0] == 1):
        scarto[-1].append(a)
        document.drop(columns=a,axis=1,inplace=True)
    else:
        continue

In [8]:
# Creo un duplicato solo per il plot 

document_plot = document

In [9]:
# Andiamo a eliminare tutti i campi dove sono presenti delle stringhe come valori
for label in document.head(0):
    document = document[document[label] != label]
    document_plot = document_plot[document_plot[label] != label]

In [10]:
# Andiamo a sostituire le label con valori interi

for b,c in zip(attaccanti_label,range(len(attaccanti_label))):
    document = document.replace(b, c)

In [11]:
# Nel documento sono presenti valori infiniti, li andiamo a sostituire con Nan che verranno successivamente rimossi

document.replace([np.inf, -np.inf], np.nan, inplace=True)
document_plot.replace([np.inf, -np.inf], np.nan, inplace=True)

In [12]:
# Eliminiamo i valori nulli

document.dropna(inplace=True)
document_plot.dropna(inplace=True)

In [13]:
# Andiamo a castare tutte le stringhe rimanenti in numeri, in quanto alcuni numeri sono rappresentati come stringhe

document = document.astype(float)
for label in document_plot.head(0):
    if(label != 'Label'):
        try:
            document_plot[label] = document_plot[label].astype(float)
        except:
            print(label)

In [14]:
# Dove presenti vanno eliminati i valori str dalle righe

'''indici = []

for a in document.head(0):
    for valore,indice in zip(document[a],document[a].index):
        if type(valore) == str:
            indici.append(indice)


if(len(indici) != 0):
    
    document.drop(indici,inplace=True,axis=0)
    document_plot.drop(indici,inplace=True,axis=0)'''

'indici = []\n\nfor a in document.head(0):\n    for valore,indice in zip(document[a],document[a].index):\n        if type(valore) == str:\n            indici.append(indice)\n\n\nif(len(indici) != 0):\n    \n    document.drop(indici,inplace=True,axis=0)\n    document_plot.drop(indici,inplace=True,axis=0)'

In [22]:
print(document.shape)

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
613099    1.0
613100    1.0
613101    0.0
613102    1.0
613103    1.0
Name: Label, Length: 607690, dtype: float64


In [16]:
# Definisco una funzione per normalizzare i valori

def normalize(feature:str):
    return (document[feature]-document[feature].mean())/np.std(document[feature])

def normalize_plot(feature:str):
    return (document_plot[feature]-document_plot[feature].mean())/np.std(document_plot[feature])


# Andiamo a normalizzare i valori per poter usare i vari algoritmi di feature selection

scaled_document = document.head(0)
scaled_document_plot = document_plot.head(0)

for a in document.head(0):
    
    try:
        if document[a].max() != 0 and a != 'Label':
            scaled_document[a] = normalize(a)
            scaled_document_plot[a] = normalize_plot(a)
        else:
            scaled_document[a] = document[a]
            scaled_document_plot[a] = document_plot[a]
    except:

        print(a)
        
        

In [17]:
# A questo punto andiamo a salvare gli histogrammi delle varie feature per effettuare analisi visive

for h in scaled_document.head(0):

    # Stampo un istogramma per ogni valori di label, così da avere la suddivisione per colore
    
    for b,c in zip(attaccanti_label,range(len(attaccanti_label))):
        scaled_document[scaled_document['Label'] == c][h].hist(label=str(b))
        
    plt.legend()
    plt.title(h)
    feature_name = h.replace('/','_')
    plt.savefig('image/'+folder+'/'+'istogrammi/'+str(feature_name)+'.png')
    plt.close()

In [18]:
# Vado a suddividere il dataset in x e y per applicare successivamente, i vari algoritmi di feature selection

x_selection = scaled_document.iloc[:,:-1]
y_selection = scaled_document.iloc[:,-1:]

In [19]:
print(y_selection.shape)
print(x_selection.isna().sum().sum())

(607690, 1)
1215380


In [20]:

# Applico il test Anova sul dataset

fvalue_Best = SelectKBest(score_func=f_classif, k=10)
fit = fvalue_Best.fit(x_selection, y_selection.astype('int'))

ValueError: Input X contains NaN.
SelectKBest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Andiamo a stampare le migliori dieci feature selezionate da Anova su un file csv

features_score = pd.DataFrame(fit.scores_)
features = pd.DataFrame(x_selection.columns)
feature_score = pd.concat([features,features_score],axis=1)

# Assegniamo un nuovo nome alle colonne

feature_score.columns = ["Input_Features","F_Score"]
print(feature_score.nlargest(10,columns="F_Score"))

In [ ]:
# Mi vado a salvare le migliori 10 feature ottenute dal test Anova con il relativo punteggio

feature_score.nlargest(10,columns="F_Score").to_csv('top_feature/'+folder+'/anova.csv',index=False)

In [ ]:
print(x_selection.head(0),y_selection.head(0))

In [ ]:
# Utilizziamo altri metodi di feature extraction per prendere le migliori 10 feature

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(x_selection, y_selection.astype('int'))

In [ ]:
features_score = pd.DataFrame(clf.feature_importances_)
features = pd.DataFrame(x_selection.columns)
feature_score = pd.concat([features,features_score],axis=1)

# Assegniamo un nuovo nome alle colonne

feature_score.columns = ["Input_Features","F_Score"]
print(feature_score.nlargest(15,columns="F_Score"))

In [ ]:
# Mi vado a salvare le migliori 10 feature ottenute dal test Anova con il relativo punteggio

feature_score.nlargest(10,columns="F_Score").to_csv('top_feature/'+folder+'/ExtraTree.csv',index=False)

In [ ]:
# Utilizziamo SVM come metodo di feature selection

features_names = x_selection.head(0)
svm = svm.SVC(kernel='linear',max_iter=400)
svm.fit(x_selection, y_selection.astype('int'))

In [ ]:
# Questa è una funziona che mi permette di visualizzare le migliori feature

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names[-10:])), imp[-10:], align='center')
    plt.yticks(range(len(names[-10:])), names[-10:])
    plt.show()


f_importances(abs(svm.coef_[0]), features_names)

In [ ]:
top_svm = pd.DataFrame()

for a in sorted(zip(abs(svm.coef_[0]),features_names),reverse=True):
    top_svm = pd.concat([top_svm, pd.DataFrame([a])], ignore_index=True)

top_svm.rename(columns = {0:'F_Score', 1:'Input_Features'}, inplace = True)
print(top_svm.head(10))

top_svm.nlargest(10,columns="F_Score").to_csv('top_feature/'+folder+'/SVM.csv',index=False)

In [ ]:
# Ultimo metodo di feature selection che utilizziamo è quello RFE, che sta per Recorsive Feature Elimination

rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=10)
# fit RFE
rfe.fit(x_selection, y_selection.astype('int'))

In [ ]:
# Andiamo a salvare le feature in un file csv

features_score = pd.DataFrame(rfe.ranking_)
features = pd.DataFrame(x_selection.columns)
feature_score = pd.concat([features,features_score],axis=1)

feature_score.columns = ["Input_Features","F_Score"]
feature_score.sort_values('F_Score',inplace=True,ascending=False,axis=0)
feature_score.nsmallest(10,columns="F_Score").to_csv('top_feature/'+folder+'/RFE.csv',index=False)

In [ ]:
# Per non creare un solo file troppo ricco di codice salviamo i dataset in formato pickle 
# per utilizzarlo successivamente in un altro file

with open('pickle/'+folder+'/scaled_document.pickle', 'wb') as handle:
    pickle.dump(scaled_document, handle, protocol=pickle.HIGHEST_PROTOCOL)
    handle.close()
with open('pickle/'+folder+'/scaled_document_plot.pickle', 'wb') as handle:
    pickle.dump(scaled_document_plot, handle, protocol=pickle.HIGHEST_PROTOCOL)
    handle.close()